In [1]:
'''6.3.2'''
import numpy as np
import pandas as pd
# Grid Search
import yfinance as yf
from sklearn.model_selection import ParameterGrid

In [2]:
df = yf.Ticker('NQ=F')
df = df.history(start='2010-01-01', end='2024-01-01')

### Rango de valores para las medias móviles

In [3]:
param_grid = {'SMA_1': [10, 20, 50],
              'SMA_2':[100, 150, 200]}

### Función de backtest para cada combinación de parámetros


In [4]:
def backtest_strategy(sma_1, sma_2):
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()
    
    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    df.dropna(inplace=True)

    #Rendimiento total de la estrategia
    return (df['Strategy_Returns'] + 1).prod() - 1

### Iteramos sobre todas las combinaciones posibles en la grid


In [5]:
best_params = None
best_return = -np.inf

In [6]:
for params in ParameterGrid(param_grid):
    sma_1 = params['SMA_1']
    sma_2 = params['SMA_2']
    total_return = backtest_strategy(sma_1, sma_2)
    print(f'Probando SMA_1: {sma_1}, SMA_2: {sma_2} - Rendimiento Total: {total_return:.2%}')

    if total_return > best_return:
        best_return = total_return
        best_params = params

print(f'Los mejores parámetros son: {best_params} con un rendimiento total de {best_return:.2%}')

Probando SMA_1: 10, SMA_2: 100 - Rendimiento Total: -29.26%
Probando SMA_1: 10, SMA_2: 150 - Rendimiento Total: 16.33%
Probando SMA_1: 10, SMA_2: 200 - Rendimiento Total: 101.77%
Probando SMA_1: 20, SMA_2: 100 - Rendimiento Total: 22.00%
Probando SMA_1: 20, SMA_2: 150 - Rendimiento Total: 99.71%
Probando SMA_1: 20, SMA_2: 200 - Rendimiento Total: 147.17%
Probando SMA_1: 50, SMA_2: 100 - Rendimiento Total: 68.16%
Probando SMA_1: 50, SMA_2: 150 - Rendimiento Total: 132.22%
Probando SMA_1: 50, SMA_2: 200 - Rendimiento Total: 137.16%
Los mejores parámetros son: {'SMA_1': 20, 'SMA_2': 200} con un rendimiento total de 147.17%
